In [34]:
import keras
keras.__version__

'2.9.0'

# Prediciendo la duración de la batería de un movil
Vamos a predecir la duración de la batería de un teléfono movil dados algunos datos como los píxeles de la cámara, si tiene o no conexión wifi, la cantidad de cores del procesador, etc. 

El conjunto de datos tiene un total de 1000 datos, que dividiremos entre 1750 ded entrenamiento (de los cuales 500 serán para validación) y 250 de test. Las 21 carácaterísticas en los datos de entrada (por ejemplo, si tiene o no v) es una característica que tiene una escala diferente. Por ejemplo, algunos valores son proporciones, que toman valores entre 0 y 1, otros toman valores entre 1 y 12, otros entre 0 y 100 ...



In [35]:
import pandas as pd
datos=pd.read_csv('train.csv.xls',encoding='latin-1')

Comprobamos que se han importado correctamente, los 2000 datos con 21 características. 

In [36]:
datos.shape

(2000, 21)

Separamos los datos en entrenamiento y test y sus correspondientes soluciones, que serán la característica 17 (talk_time) que según explicaba la página donde hemos tomado la base, es el tiempo que dura el móvil desde su última carga. 


In [37]:
labels=datos.talk_time
labels.shape


(2000,)

In [38]:
datos=datos.drop(['talk_time'],axis=1)

In [39]:
train_datos=datos[:1749]
test_datos=datos[1750:]

train_labels=labels[:1749]
test_labels=labels[1750:]

Una vez tenemos los datos separados, vamos a normalizarlos, puesto que cada uno tienen rangos distintos. 

In [40]:
# Normalizamos basándonos en la característica
##############################################
# Calculamos la media del primer eje de los datos de entrenamiento
mean = train_datos.mean(axis=0)
print("##### Media de las 21 características de los datos de entrenamiento ##### ")
print(mean)
# A los datos de entrenamiento le restamos la media
train_datos -= mean

# Calculamos la desviación estandar del primer eje de los datos de entrenamiento
std = train_datos.std(axis=0)
print("##### Desviación estandar de las 21 características de los datos de entrenamiento ##### ")
print(std)
# A los datos de entrenamiento los dividimos por la desviación estándar
train_datos /= std

# ¡¡IMPORTANTE!!: USAMOS LA MEDIA Y LA DESVIACIÓN ESTANDAR DE LOS DATOS DE 
# ENTRENAMIENTO PARA NORMALIZAR EL SET DE DATOS DE LOS DATOS DE TEST 
test_datos -= mean
test_datos /= std

##### Media de las 21 características de los datos de entrenamiento ##### 
battery_power    1243.436821
blue                0.485420
clock_speed         1.523899
dual_sim            0.509434
fc                  4.357347
four_g              0.523728
int_memory         32.207547
m_dep               0.500515
mobile_wt         140.978273
n_cores             4.507719
pc                  9.915380
px_height         648.325901
px_width         1253.442539
ram              2132.694683
sc_h               12.359634
sc_w                5.802173
three_g             0.763293
touch_screen        0.502573
wifi                0.508862
price_range         1.514580
dtype: float64
##### Desviación estandar de las 21 características de los datos de entrenamiento ##### 
battery_power     438.149513
blue                0.499930
clock_speed         0.820232
dual_sim            0.500054
fc                  4.378650
four_g              0.499580
int_memory         18.098062
m_dep               0.287984
mobile_wt

## CREAMOS LA RED

Como tenemos una base de datos tan pequeña, vamos a crear una base de datos muy pequeña con únicamente dos capas ocultas. 
 En general, cuantos menos datos de entrenamiento tengamos, peor será el sobreajuste y el uso una red pequeña es una forma de reducir el sobreajuste.

In [41]:
import tensorflow as tf
from keras import models
from keras import layers

def build_model():
    # Porque necesitaremos instanciar el mismo modelo 
    # múltiples veces, usamos una función para construirlo
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_datos.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    # MSE: Mean Squared Error
    # MAE: Mean Absolute Error
    model.compile(optimizer='rmsprop', loss='mse', 
                  metrics=[tf.keras.metrics.MeanAbsoluteError()]) # metrics = ['mae']
    return model

Con estos datos, computamos el problema con las primeras 3/4 partes de los datos de entrenamiento para entrenar y el último 1/4 de los datos para validación

In [42]:
partial_train_datos = train_datos[:1499]
partial_eval_train_datos = train_datos[1500:]
partial_train_labels = train_labels[:1499]
partial_eval_train_labels= train_labels[1500:]

In [43]:
mymodel = build_model()

In [44]:
history = mymodel.fit(partial_train_datos, partial_train_labels,
            validation_data=(partial_eval_train_datos, partial_eval_train_labels),
                        epochs=40, batch_size=1)

Epoch 1/40
1499/1499 [==============================] - 3s 2ms/step - loss: 39.5040 - mean_absolute_error: 5.2369 - val_loss: 36.6758 - val_mean_absolute_error: 5.1879
Epoch 2/40
1499/1499 [==============================] - 2s 1ms/step - loss: 30.8687 - mean_absolute_error: 4.7379 - val_loss: 34.5799 - val_mean_absolute_error: 5.0302
Epoch 3/40
1499/1499 [==============================] - 3s 2ms/step - loss: 30.1036 - mean_absolute_error: 4.6538 - val_loss: 35.0189 - val_mean_absolute_error: 5.0494
Epoch 4/40
1499/1499 [==============================] - 2s 1ms/step - loss: 29.2522 - mean_absolute_error: 4.6214 - val_loss: 35.5342 - val_mean_absolute_error: 5.1190
Epoch 5/40
1499/1499 [==============================] - 2s 1ms/step - loss: 28.7376 - mean_absolute_error: 4.5383 - val_loss: 35.5476 - val_mean_absolute_error: 5.1582
Epoch 6/40
1499/1499 [==============================] - 2s 1ms/step - loss: 27.9017 - mean_absolute_error: 4.5001 - val_loss: 37.1157 - val_mean_absolute_error:

In [ ]:
mae_history = history.history['val_mean_absolute_error']

In [ ]:
mse_history = history.history['val_loss']

Confrontamos en una gráfica las épocas frente a la pérdida pérdida (MSE)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(mse_history) + 1), mse_history)
plt.xlabel('Epochs')
plt.ylabel('Validación MSE')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Pérdida del entreno')
plt.plot(epochs, val_loss, 'b', label='Pérdida de la validación')
plt.title('Perdidas del entreno y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

plt.show()

Confrontamos en una gráfica las épocas contra la métrica de error absoluto medio (MAE)

In [ ]:
 import matplotlib.pyplot as plt

plt.plot(range(1, len(mae_history) + 1), mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validación MAE')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['mean_absolute_error']
val_loss = history.history['val_mean_absolute_error']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Pérdida del entreno')
plt.plot(epochs, val_loss, 'b', label='Pérdida de la validación')
plt.title('Perdidas del entreno y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

plt.show()